# Code from chapter 2 of "Statistical Rethinking," 2nd edition.

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import scipy.stats as stats
import seaborn as sns

## Computing plausibilities (R code 2.1)

In [ ]:
ways = np.array([0, 3, 8, 9, 0])
ways / sum(ways)

The plausibilities above are probabilities. They total to 1.

In [ ]:
sum(Out[2])

The design loop for simple Bayesian models has three steps.

1. Data story: Motivate the model by narrating **how** the data
   might arise.
2. Update: Educate the model by feeding it the (observed) data.
3. Evaluate: All statistical models require supervision,
   leading to model revision.

In the globe case, the data story is simply a restatement of the
sampling process.

1. The true proportion of the water covering the globe is _p_.
2. A single toss of the globe has a probability, _p_, of 
   producing a water (_W_) observation. It has a probability,
   _1 - p_, of producing a land (_L_) observation.
3. Each toss of the globe is independent of the others.

In [ ]:
data = ['W', 'L', 'W', 'W', 'W', 'L', 'W', 'L', 'W']

In [ ]:
fig, axes = plt.subplots(3, 3, sharex=True)
prior_alpha, prior_beta = 0, 0
posterior_alpha, posterior_beta = 0, 0
for i in range(len(axes)):
    for j in range(len(axes[0])):
        xs = np.linspace(stats.beta.ppf(0.01, prior_alpha, prior_beta),
                         stats.beta.ppf(0.99, prior_alpha, prior_beta))
        axes[i, j].plot(xs, stats.beta.pdf(xs, prior_alpha, prior_beta), '--')
        axes[i, j].set_title(f'{i}, {j}')
        if data[i * 3 + j] == 'W':
            posterior_alpha += 1
        else:
            posterior_beta += 1
        axes[i, j].plot(xs, stats.beta.pdf(xs, posterior_alpha, posterior_beta), '--')
        
        prior_alpha, prior_beta = posterior_alpha, posterior_beta